In [1]:
# imports and constancts omitted for brevity

In [ ]:
# Densenet CNNs are: BN-ReLU-Conv2D
inputs = Input(shape=input_shape)
X = BatchNormalization()(inputs)
X = Activation('relu')(X)
X = Conv2D(n_filters_before_dense_block,
           kernel_size=3,
           padding='same',
           kernel_initializer='he_normal')(X)
X = concatenate([inputs, X])

# Stack of Dense blocks bridged by transition layers
for i in range(n_dense_blocks):
    # Dense block is a stack of bottleneck layers
    for j i n range(n_bottleneck_layers):
        y = BatchNormalization()(X)
        y = Activation('relu')(y)
        y = Conv2D(4 * growth_rate,
                   kernel_size=1,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        y = BatchNormalization()(y)
        y = Activation('relu')(y)
        y = Conv2D(growth_rate,
                   kernel_size=3,
                   padding='same',
                   kernel_initializer='he_normal')(y)
        if not data_augmentation:
            y = Dropout(0.2)(y)
        X = concatenate([X, y])
        
    # No transition layer after last dense block
    if i == n_dense_blocks - 1:
        continue
        
    # Transition layer compresses n feature maps and reduces size by 2
    n_filters_before_dense_block += n_bottleneck_layers * growth_rate
    n_filters_before_dense_block = int(n_filters_before_dense_block 
                                       * compression_factor)
    y = BatchNormalization()(X)
    y = Conv2D(n_filters_before_dense_block,
               kernel_size=1,
               padding='same',
               kernel_initializer='he_normal')(y)
    if not data_augmentation:
        y = Dropout(0.2)(y)
    x = AveragePooling2D()(y)
    
# Add classifier on top
# After pooling size of feature map is 1x1
X = AveragePooling2D(pool_size=8)(X)
y = Flatten()(X)
outputs = Dense(
    n_classes, kernel_initializer='he_normal', activation='softmax'
)(y)

# Instantiate and compile mod
# original paper uses SGD but RMSprop works better
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(1e-3), 
              metrics=['accuracy'])
model.summary()